In [19]:
import random
import networkx as nx
import itertools

In [20]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [21]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [22]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [23]:
def fitness(resolving_set,G,shortest_paths):
    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [24]:
def construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths):
    resolving_set = set()
    available_nodes = set(G.nodes)

    while available_nodes:
        # Računanje verovatnoće izbora svakog čvora
        probabilities = {node: (pheromones[node] ** alpha) * (heuristics[node] ** beta) for node in available_nodes}
        total = sum(probabilities.values())
        
        if total == 0:
            break
        
        # Normalizacija verovatnoća
        probabilities = {node: prob / total for node, prob in probabilities.items()}

        # Izbor čvora proporcionalno verovatnoći
        selected_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()))[0]
        resolving_set.add(selected_node)
        available_nodes.remove(selected_node)

        # Ako je rešenje dovoljno dobro, prekidamo ranije
        if fitness(resolving_set,G,shortest_paths) < 100:
            break

    return resolving_set

In [25]:
def update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths):
    # Isparavanje feromona
    for node in pheromones:
        pheromones[node] *= (1 - evaporation)
    
    # Dodavanje novih feromona na čvorove iz najboljeg rešenja
    for node in best_solution:
        pheromones[node] += Q / fitness(best_solution,G,shortest_paths)

In [26]:
def ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths):
    best_solution = None
    best_fitness = float("inf")

    for iteration in range(num_iterations):
        solutions = [construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths) for _ in range(num_ants)]
        sorted_solutions = sorted(solutions, key=lambda s: fitness(s, G,shortest_paths))        
        if fitness(sorted_solutions[0],G,shortest_paths) < best_fitness:
            best_solution = sorted_solutions[0]
            best_fitness = fitness(best_solution,G,shortest_paths)

        update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths)
        
        print(f"Iteracija {iteration}, najbolji skup {best_fitness},  {best_solution}")
    print("\nOptimalni rešavajući skup:", best_solution, 'velicine:',len(best_solution))
    return best_solution

In [27]:
# G = load_graph('grafovi\srednji grafovi\hamming8-4.txt')
G = load_gcol('grafovi\gcol\gcol1.txt')

n = len(G.nodes)

# Parametri ACO algoritma
num_ants = 50     # Broj mrava po iteraciji
num_iterations = 50  # Broj iteracija
alpha = 0.5       # Uticaj feromona
beta = 1.0        # Uticaj heuristike (udaljenosti)
evaporation = 0.5  # Isparavanje feromona
Q = 10          # Konstanta za ažuriranje feromona

# Inicijalizacija feromona (svaki čvor dobija početnu vrednost)
pheromones = {node: 1.0 for node in G.nodes}

# heuristics = {node: 1.0 / (sum(nx.shortest_path_length(G, node, j) for j in G.nodes if j != node) + 1) for node in G.nodes}

centrality = nx.betweenness_centrality(G)
heuristics = {node: centrality[node] for node in G.nodes}

shortest_paths = dict(nx.all_pairs_shortest_path_length(G))  # Svi putevi između svih parova


rez = ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths)


<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_171612\1022156341.py:2: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol1.txt')


Iteracija 0, najbolji skup 11,  {97, 98, 100, 69, 5, 12, 14, 17, 86, 57, 95}
Iteracija 1, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 2, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 3, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 4, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 5, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 6, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 7, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 8, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 9, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 10, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 11, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 12, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija 13, najbolji skup 10,  {97, 1, 37, 8, 9, 76, 49, 52, 22, 56}
Iteracija

KeyboardInterrupt: 

In [ ]:
is_resolving_set(rez,G)